In [1]:
import numpy as np
import typing
import matplotlib.pylab as plt

In [2]:
def plot_func(func, ylim=True):
    x = np.arange(-5.0, 5.0, 0.1)
    y = func(x)
    plt.plot(x, y)
    if ylim:
        plt.ylim(-0.1, 1.1, emit=False)
        plt.show()
    else:
        plt.show()

# 阶跃函数
$u(t)=\epsilon(t)=\begin{cases}0,&t<0\\1,&t>0\end{cases}$

In [3]:
#类型提示
def step_function(x:np.ndarray):
    #进行不等式运算, y是一个布尔类型数组
    y = x > 0
    return np.array(x > 0, dtype = int)

plot_func(step_function)

# sigmoid 函数
$f(t) = {1\over{1+e^{-x}}}$

In [4]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

plot_func(sigmoid)

# ReLU函数(Rectified Linear Unit)
$u(t)=\begin{cases}t,&t>0\\0,&t\leq 0\end{cases}$

In [5]:
def ReLU(x):
    return np.maximum(0, x)

plot_func(ReLU,ylim = False)

In [6]:
def identity_function(X):
    return X

# 简单神经网络的实现
<img src="https://picstore-of-ambi.oss-cn-shanghai.aliyuncs.com/img/20240503002947.png" alt="image.png" style="float: left;">

In [7]:
class Layer:
    def __init__(self, W, B):
        # X.size = input_size
        # W.size = nextlayer.size
        # B.size = nextlayer.szie
        self.W = W
        self.B = B

    def forward(self, X):
        return np.dot(X, self.W) + self.B
    
    def __call__(self, X):
        return self.forward(X)

class Network:
    def __init__(self):
        self.W1 = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]]) 
        self.B1 = np.array([0.1, 0.2, 0.3])
        self.W2 = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
        self.B2 = np.array([0.1, 0.2])
        self.layer1 = Layer(self.W1, self.B1)
        self.layer2 = Layer(self.W2, self.B2)
    
    def forward(self, X):
        X1 = sigmoid(self.layer1(X))
        X2 = sigmoid(self.layer2(X1))
        return identity_function(X2)

    def __call__(self, X):
        return self.forward(X)
    
X = np.array([1.0, 0.5]) 
net = Network()
output = net(X)
print(output)

In [8]:
def softmax(X):
    C = np.max(X)
    exp_a = np.exp(X - C) #防止溢出
    sum_exp_a = np.sum(exp_a)
    
    return exp_a / sum_exp_a

# MNIST简单实战


In [9]:
# 读入数据
import sys, os
from dataset.mnist import load_mnist
from PIL import Image

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

# 输出各个数据的形状 
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000,) 
print(x_test.shape) # (10000, 784) 
print(t_test.shape) # (10000,) 

img = x_train[0].reshape(28,28) # 图像原来为28*28
label = t_train[0]
print(label)

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

img_show(img)

<img src="https://picstore-of-ambi.oss-cn-shanghai.aliyuncs.com/img/20240503141242.png" alt="image.png" style="float: left;">

In [12]:
import pickle

with open(r"./utils/sample_weight.pkl", "rb") as f:
    weight = pickle.load(f)


class MNIST_Network():
    def __init__(self):
        self.W1, self.W2, self.W3 = weight['W1'], weight['W2'], weight['W3']
        self.b1, self.b2, self.b3 = weight['b1'], weight['b2'], weight['b3']
        self.layer1 = Layer(self.W1, self.b1)
        self.layer2 = Layer(self.W2, self.b2)
        self.layer3 = Layer(self.W3, self.b3)

    def forward(self, X):
        X1 = self.layer1(X)
        X2 = self.layer2(X1)
        X3 = self.layer3(X2)
        return softmax(X3)

    def __call__(self, X):
        return self.forward(X)


batch_size = 100
accuracy_cnt = 0
net = MNIST_Network()
for i in range(0, len(x_test), batch_size):
    x_batch = x_test[i:i + batch_size]
    y_batch = net(x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t_test[i:i + batch_size])

print("Accuracy: " + str(float(accuracy_cnt / len(x_test))))